# Supplementary Notebook: Skyrmion U-Net Analysis Editor for Local Files

This notebook demonstrates how to perform an analysis using the U-Net results with an editor similar to the one in `3_Editor.ipynb`. The key difference is that this notebook is designed to handle **large local files** more efficiently. Unlike the `3_Editor.ipynb` notebook, data is not temporarily loaded into the memory of the current notebook session, which makes it better suited for larger datasets. This notebook can also be ideally used after the hands-on session.

## 0. Configure the notebook

In [ ]:
try:
    import google.colab
    in_colab = True
    ![ ! -d "AI-Magnetism-Session-Regensburg-2025" ] ||  [ ! -d "AI-Magnetism-Session-Regensburg-2025/.git" ] && git clone https://github.com/kfjml/AI-Magnetism-Session-Regensburg-2025
    ! pip install "numba>=0.61.0,<0.62" "tensorflow[and-cuda]>=2.16.2,<3" "albumentations>=2.0.4,<3"  "pandas>=2.2.2,<3" "chardet>=5.2.0,<6" "opencv-python-headless>=4.11.0.86,<5" "wget>=3.2,<4" "pyyaml>=6.0.2, <7" "pillow>=11.1.0, <12"
    ! pip install "ipympl>=0.9.6" "ipywidgets>=7.7.1" "matplotlib>=3.10.0,<4"
    basis_dir = "/content/AI-Magnetism-Session-Regensburg-2025/"
    import sys
    sys.path.append(basis_dir)

    from google.colab import output
    output.enable_custom_widget_manager()
except:
    basis_dir = "./"    
    in_colab = False

from UNetEditor import UNetGUI
from UNetEditor import get_video_analysis,get_batch_analysis
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import pandas as pd
import os

basis_dir = "./"         
tmp_folder,zip_folder,result_file="./tmp/","zipfolder/","result"
models = [(basis_dir+'models/2023_model.keras',"Model 2023",2),(basis_dir+'models/2022_model.keras',"Model 2022",1),(basis_dir+'models/2022_model_inv.keras',"Model 2022 inverse",1)]  

### If you are running this notebook in **Google Colab**, after executing the first cell (cell above), go to **Runtime → Restart session**, then rerun the first cell. After that, you can execute the cells below. This is necessary because some required packages are installed in Google Colab and need a restart to take effect.

## 1. Process Local Video File

Load the first frame of a video to set the settings in the editor, make a sample prediction, and configure the analysis settings. After that, the entire set of frames will be predicted.  In the section below this code, there is an example for processing a batch of local images.

In [ ]:
def get_frame(video_file):
    import cv2
    cap = cv2.VideoCapture(video_file)
    ret,frame = cap.read()
    if not ret: 
        print("Error: ",video_file)
        return None
    cap.release()
    return frame

video_file = "../test.avi"#change this file name to the local filepath of the video
frame = get_frame(video_file)

Set the settings in the editor, make a sample prediction with the first frame, and configure the analysis settings using the first frame.

In [ ]:
plt.close("all")
%matplotlib widget  
gui = UNetGUI(models,frame,tmp_folder,zip_folder,result_file,change_img=False,colab=in_colab)
gui

Now, with these settings/configurations, predict the entire video. 

In [ ]:
config = gui.get_config()
if not ((config["mask_analysis_editor"] is None) or (config["pos_analysis_editor"] is None)):
    file = get_video_analysis(video_file,config,tmp_folder+zip_folder,tmp_folder+result_file)
    print("Result saved in ",file)
else:
    print("Set the parameters in the GUI ! (min. click predict)")

Check the resulting zip file by loading its contents.

In [ ]:
arch = zipfile.ZipFile(file,"r")
print(file,arch.namelist())
lcsv = sorted([ele for ele in arch.namelist() if ele.endswith(".csv")])
lpng = sorted([ele for ele in arch.namelist() if ele.endswith(".png")])

Examine one prediction and the corresponding CSV file for a single frame.

In [ ]:
plt.close("all")
%matplotlib inline  

ix = 0
print(lcsv[ix],lpng[ix])
fig,ax = plt.subplots()
ax.imshow(plt.imread(arch.open(lpng[ix],"r")))
pd.read_csv(arch.open(lcsv[ix],"r"))

## 2. Process Local Batch of Images

Load the batch of images.

In [ ]:
import glob
image_filename_list = ["test.png","test1.png"]
image_list = [(np.array(Image.open(ele)),os.path.basename(ele)) for ele in  image_filename_list]

Set the settings in the editor, make a sample prediction, and configure the analysis settings for a single image.

In [ ]:
plt.close("all")
%matplotlib widget  
gui = UNetGUI(models,image_list[0][0],tmp_folder,zip_folder,result_file,change_img=False,colab=in_colab)
gui

Now, with these settings/configurations, predict the entire batch of images. 

In [ ]:
config = gui.get_config()
if not ((config["mask_analysis_editor"] is None) or (config["pos_analysis_editor"] is None)):
    file = get_batch_analysis(image_list,config,tmp_folder+zip_folder,tmp_folder+result_file)
    print("Result saved in ",file)
else:
    print("Set the parameters in the GUI ! (min. click predict)")

Check the resulting zip file by loading its contents.

In [ ]:
arch = zipfile.ZipFile(file,"r")
print(file,arch.namelist())
lcsv = sorted([ele for ele in arch.namelist() if ele.endswith(".csv")])
lpng = sorted([ele for ele in arch.namelist() if ele.endswith(".png")])

Examine one prediction and the corresponding CSV file for a single image.

In [ ]:
plt.close("all")
%matplotlib inline  

ix = 0
print(lcsv[ix],lpng[ix])
fig,ax = plt.subplots()
ax.imshow(plt.imread(arch.open(lpng[ix],"r")))
pd.read_csv(arch.open(lcsv[ix],"r"))